# Whole Code

```
import boto3 

REGION = 'us-east-1'

#list all sg 
#find all attched sg

#filter 

ec2 = boto3.client('ec2', region_name=REGION)
def get_all_security_groups() -> list:
    return [sg.get('GroupId')for sg in ec2.describe_security_groups().get('SecurityGroups')]

def get_attached_security_groups() -> list:
    base = ec2.describe_instances().get('Reservations')
    return [i.get('Instances')[0]['SecurityGroups'][0].get('GroupId') for i in base]

#lambda 'group' (this is the input for the anon function) : this is the condition  group not in attached_sg_list 
#filter takes an iterable (list) as the second param and evaluates each against lambda condition 
#checks to see if sg from all listings is in attached 
def filter_unattached_sg() -> list:
    return list(filter(lambda group: group not in get_attached_security_groups(), get_all_security_groups()))
  
def delete_unattached_sg(unattached_list):
    for sg in unattached_list:
        try:
            ec2.delete_security_group(GroupId=sg)
        except Exception as e:
            if '"default" cannot be deleted by a user' in str(e):
                print("Unable to delete default security group")

def init():
    delete_unattached_sg(filter_unattached_sg())



if __name__ == "__main__":
    init()

```
#Elliott Arnold boto3 practice delete sg not attached to EC2 instance  
#5-27-22 

In [5]:
import boto3 

REGION = 'us-east-1'

#list all sg 
#find all attched sg
#filter 

ec2 = boto3.client('ec2', region_name=REGION)

## ----- Steps for get_all_security_groups()

In [7]:
ec2.describe_security_groups()

{'SecurityGroups': [{'Description': 'launch-wizard-4 created 2022-03-12T10:44:00.601-05:00',
   'GroupName': 'launch-wizard-4',
   'IpPermissions': [{'FromPort': 22,
     'IpProtocol': 'tcp',
     'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'ToPort': 22,
     'UserIdGroupPairs': []},
    {'FromPort': 3000,
     'IpProtocol': 'tcp',
     'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
     'Ipv6Ranges': [{'CidrIpv6': '::/0'}],
     'PrefixListIds': [],
     'ToPort': 3000,
     'UserIdGroupPairs': []}],
   'OwnerId': '027940810740',
   'GroupId': 'sg-0b9ae95a557e9e597',
   'IpPermissionsEgress': [{'IpProtocol': '-1',
     'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'UserIdGroupPairs': []}],
   'VpcId': 'vpc-0ea73b2740360fb3a'},
  {'Description': 'launch-wizard-2 created 2022-05-27T20:52:01.487Z',
   'GroupName': 'launch-wizard-2',
   'IpPermissions': [{'FromPort': 22,
     'IpProtocol': 'tcp',
     

In [8]:
ec2.describe_security_groups().get('SecurityGroups')

[{'Description': 'launch-wizard-4 created 2022-03-12T10:44:00.601-05:00',
  'GroupName': 'launch-wizard-4',
  'IpPermissions': [{'FromPort': 22,
    'IpProtocol': 'tcp',
    'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
    'Ipv6Ranges': [],
    'PrefixListIds': [],
    'ToPort': 22,
    'UserIdGroupPairs': []},
   {'FromPort': 3000,
    'IpProtocol': 'tcp',
    'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
    'Ipv6Ranges': [{'CidrIpv6': '::/0'}],
    'PrefixListIds': [],
    'ToPort': 3000,
    'UserIdGroupPairs': []}],
  'OwnerId': '027940810740',
  'GroupId': 'sg-0b9ae95a557e9e597',
  'IpPermissionsEgress': [{'IpProtocol': '-1',
    'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
    'Ipv6Ranges': [],
    'PrefixListIds': [],
    'UserIdGroupPairs': []}],
  'VpcId': 'vpc-0ea73b2740360fb3a'},
 {'Description': 'launch-wizard-2 created 2022-05-27T20:52:01.487Z',
  'GroupName': 'launch-wizard-2',
  'IpPermissions': [{'FromPort': 22,
    'IpProtocol': 'tcp',
    'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
    'Ipv

In [11]:
[sg.get('GroupId') for sg in ec2.describe_security_groups().get('SecurityGroups')]

['sg-0b9ae95a557e9e597',
 'sg-0835e10401540eaf0',
 'sg-04136d3c40abbae9e',
 'sg-0ab6bec290e808f90',
 'sg-081f6c1a9d9d8cec0',
 'sg-0ef03ce9260622651']

In [32]:
def get_all_security_groups() -> list:
    return [sg.get('GroupId')for sg in ec2.describe_security_groups().get('SecurityGroups')]

In [33]:
get_all_security_groups()

['sg-0b9ae95a557e9e597',
 'sg-0835e10401540eaf0',
 'sg-04136d3c40abbae9e',
 'sg-0ab6bec290e808f90',
 'sg-081f6c1a9d9d8cec0',
 'sg-0ef03ce9260622651']

## ----- Steps for get_attached_security_groups()

In [16]:
base = ec2.describe_instances().get('Reservations')
display(base)

[{'Groups': [],
  'Instances': [{'AmiLaunchIndex': 0,
    'ImageId': 'ami-04505e74c0741db8d',
    'InstanceId': 'i-076a1b73343c764b3',
    'InstanceType': 't2.micro',
    'KeyName': 'Project2InstanceKeyPair',
    'LaunchTime': datetime.datetime(2022, 3, 12, 19, 22, 8, tzinfo=tzutc()),
    'Monitoring': {'State': 'disabled'},
    'Placement': {'AvailabilityZone': 'us-east-1c',
     'GroupName': '',
     'Tenancy': 'default'},
    'PrivateDnsName': 'ip-172-31-82-10.ec2.internal',
    'PrivateIpAddress': '172.31.82.10',
    'ProductCodes': [],
    'PublicDnsName': '',
    'State': {'Code': 80, 'Name': 'stopped'},
    'StateTransitionReason': 'User initiated (2022-03-12 22:33:57 GMT)',
    'SubnetId': 'subnet-0561535e87be09b43',
    'VpcId': 'vpc-0ea73b2740360fb3a',
    'Architecture': 'x86_64',
    'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
      'Ebs': {'AttachTime': datetime.datetime(2022, 3, 12, 19, 22, 9, tzinfo=tzutc()),
       'DeleteOnTermination': True,
       'Status': '

In [19]:
[i.get('Instances')[0] for i in base]

[{'AmiLaunchIndex': 0,
  'ImageId': 'ami-04505e74c0741db8d',
  'InstanceId': 'i-076a1b73343c764b3',
  'InstanceType': 't2.micro',
  'KeyName': 'Project2InstanceKeyPair',
  'LaunchTime': datetime.datetime(2022, 3, 12, 19, 22, 8, tzinfo=tzutc()),
  'Monitoring': {'State': 'disabled'},
  'Placement': {'AvailabilityZone': 'us-east-1c',
   'GroupName': '',
   'Tenancy': 'default'},
  'PrivateDnsName': 'ip-172-31-82-10.ec2.internal',
  'PrivateIpAddress': '172.31.82.10',
  'ProductCodes': [],
  'PublicDnsName': '',
  'State': {'Code': 80, 'Name': 'stopped'},
  'StateTransitionReason': 'User initiated (2022-03-12 22:33:57 GMT)',
  'SubnetId': 'subnet-0561535e87be09b43',
  'VpcId': 'vpc-0ea73b2740360fb3a',
  'Architecture': 'x86_64',
  'BlockDeviceMappings': [{'DeviceName': '/dev/sda1',
    'Ebs': {'AttachTime': datetime.datetime(2022, 3, 12, 19, 22, 9, tzinfo=tzutc()),
     'DeleteOnTermination': True,
     'Status': 'attached',
     'VolumeId': 'vol-02065c982220b3baf'}}],
  'ClientToken': ''

In [24]:
[i.get('Instances')[0]['SecurityGroups'][0] for i in base]

[{'GroupName': 'launch-wizard-4', 'GroupId': 'sg-0b9ae95a557e9e597'},
 {'GroupName': 'launch-wizard-2', 'GroupId': 'sg-0835e10401540eaf0'},
 {'GroupName': 'launch-wizard-1', 'GroupId': 'sg-0ab6bec290e808f90'}]

In [27]:
[i.get('Instances')[0]['SecurityGroups'][0].get('GroupId') for i in base]

['sg-0b9ae95a557e9e597', 'sg-0835e10401540eaf0', 'sg-0ab6bec290e808f90']

In [29]:
def get_attached_security_groups() -> list:
    base = ec2.describe_instances().get('Reservations')
    return [i.get('Instances')[0]['SecurityGroups'][0].get('GroupId') for i in base]

In [34]:
get_attached_security_groups()

['sg-0b9ae95a557e9e597', 'sg-0835e10401540eaf0', 'sg-0ab6bec290e808f90']

In [42]:
get_all_security_groups()

['sg-0b9ae95a557e9e597',
 'sg-0835e10401540eaf0',
 'sg-0ab6bec290e808f90',
 'sg-081f6c1a9d9d8cec0']

# -----------------------

## ----- Steps for filter_unattached_sg()

In [43]:
def filter_unattached_sg() -> list:
    return list(filter(lambda group: group not in get_attached_security_groups(), get_all_security_groups()))


In [41]:
filter_unattached_sg()

['sg-081f6c1a9d9d8cec0']

# -----------------------

In [ ]:
if you run this command below it will delete your unsattached SGs

In [ ]:
# def delete_unattached_sg(unattached_list):
# for sg in unattached_list:
#     try:
#         ec2.delete_security_group(GroupId=sg)
#     except Exception as e:
#         if '"default" cannot be deleted by a user' in str(e):
#             print("Unable to delete default security group")


In [ ]:
# def init():
#     delete_unattached_sg(filter_unattached_sg())



# if __name__ == "__main__":
#     init()